In [6]:
%pip install -q kaggle
%pip install -q kagglehub
%pip install -q dask
%pip install -q ipywidgets --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd, requests
import kagglehub, re, json, os
import requests, subprocess
import sys, time
from tqdm import tqdm

In [8]:
# Définis le chemin du fichier kaggle.json
path = r"C:\Credentials\kaggle.json"
#json file : {"username":"flosrv","api_key":"44c79a4d5d1534092d24ee29f34e6ee3"}
# Charge les credentials depuis le fichier JSON
with open(path, 'r') as file:
    content = json.load(file)

# Récupère les valeurs du fichier
username = content["username"]
api_key = content["key"]  # ⚠️ Assure-toi que la clé s'appelle bien "key" dans ton fichier JSON

# ✅ Définis les variables d'environnement avant de les utiliser
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = api_key
import kaggle
from kaggle import KaggleApi
from kaggle import api # import the already authenticated API client
# 1. **Authentification avec Kaggle**
def authenticate_kaggle():
    # Vérifie si le fichier kaggle.json est dans le bon dossier
    kaggle_json_path = os.path.expanduser(path)  # pour Linux/macOS
    # Pour Windows, utilisez : os.path.expanduser(r'C:\Users\<Votre-Nom>\.kaggle\kaggle.json')

    # Si le fichier n'existe pas, avertir l'utilisateur
    if not os.path.exists(kaggle_json_path):
        print("Le fichier 'kaggle.json' n'est pas trouvé. Téléchargez-le depuis votre compte Kaggle.")
        print("Assurez-vous de le placer dans le répertoire ~/.kaggle/ (Linux/macOS) ou C:\\Users\\<Your-Username>\\.kaggle\\ (Windows).")
        return False  # L'authentification échoue
    else:
        print(f"Authentification réussie avec le fichier '{kaggle_json_path}'")
        api = KaggleApi()
        api.authenticate()  # Authentification avec le fichier 'kaggle.json'
        return True

try:
    # Exécuter l'authentification
    authenticate_kaggle()
except Exception as e:
    print(f"Erreur lors de l'authentification : {e}")
    sys.exit(1)

Authentification réussie avec le fichier 'C:\Credentials\kaggle.json'


In [9]:
path = r"C:\Users\f.gionnane\Downloads\Mes-Projets\Kaggle Data Projects\data"

Functions For Datasets

In [ ]:
def list_datasets(keyword=None, usability_min=None, min_mb=None, columns=None):
    print(f"Lister les datasets correspondant à '{keyword if keyword else 'tous les datasets'}' :")
    
    """
    dataset metadata:
    'id', 'ref', 'subtitle', 'creatorName', 'creatorUrl', 'totalBytes',
        'url', 'lastUpdated', 'downloadCount', 'licenseName', 'description',
        'ownerName', 'ownerRef', 'kernelCount', 'title', 'viewCount',
        'voteCount', 'currentVersionNumber', 'usabilityRating', 'tags'
    """
    
    # Traitement des mots-clés
    if keyword:
        if isinstance(keyword, str):
            keyword = [keyword]
        keyword = [re.escape(kw) for kw in keyword]

    # Recherche des datasets via l'API Kaggle
    datasets = []
    for kw in keyword:
        datasets.extend(api.dataset_list(search=kw))

    result = []
    for dataset in datasets:
        dataset_dict = dataset.to_dict()  # Convertir le dataset en dictionnaire
        
        title = dataset_dict.get("title", "")
        ref = dataset_dict.get("ref", "")
        description = dataset_dict.get("description", "")
        lastUpdated = dataset_dict.get("lastUpdated", "")
        voteCount = dataset_dict.get("voteCount", 0)
        tags = dataset_dict.get("tags", [])
        usabilityRating = dataset_dict.get("usabilityRating", 0)
        size = dataset_dict.get("totalBytes", 0)

        # Convertir la taille en Mo
        dataset_size = dataset_dict.get("totalBytes", 0)
        dataset_size_mb = dataset_size / (1024 * 1024)

        # Filtrage selon la taille du dataset
        if min_mb is not None and dataset_size_mb < min_mb:
            continue

        # Filtrage selon la note d'utilisabilité
        if usability_min is not None and usabilityRating < usability_min:
            continue

        # Si des colonnes spécifiques sont demandées
        if columns is not None:
            dataset_dict = {key: dataset_dict[key] for key in columns if key in dataset_dict}

        # Ajouter les informations dans le résultat
        result_dict = {
            "title": title,
            "ref": ref,
            "description": description,
            "lastUpdated": lastUpdated,
            "voteCount": voteCount,
            "tags": tags,
            "usabilityRating": usabilityRating,
            "dataset_size_mb": round(dataset_size_mb,2),
            "Size (Mb)" : round(dataset_dict.get("totalBytes", 0) / (1024 * 1024),2)  # Conversion en Mo
        }

        result.append(result_dict)

    # Création du DataFrame
    df = pd.DataFrame(result)
    
    
    print(f"\n{df.shape[0]} datasets trouvés.")
    return df

keywords = [
    "sales",          # Ventes : analyses de performance commerciale
    "marketing",      # Campagnes, ROI, funnel, etc.
    "customer",       # Données clients : segmentation, churn, satisfaction
    "finance",        # Budgets, prévisions, états financiers
    "supply chain",   # Logistique, stocks, prévision de demande
    "human resources",# RH : turnover, performance, recrutement
    "retail",         # Analyse de produits, points de vente, inventaire
    "ecommerce",      # Données web, comportement d'achat, panier moyen
    "operations",     # Optimisation de processus internes
    "real estate"     # Immobilier : investissements, prévisions de prix
]

df_consulting_datasets =list_datasets(keyword=keywords, usability_min=0.6, min_mb=10)

Lister les datasets correspondant à '['sales', 'marketing', 'customer', 'finance', 'supply chain', 'human resources', 'retail', 'ecommerce', 'operations', 'real estate']' :

39 datasets trouvés.


In [11]:
agriculture_keywords = ["agriculture", "food", "farming", "vegetable", "livestock", "dairy", "poultry", "meat", "fruit", "fruits","nuts", "seeds", 
                        "grains", "cereals"]

df_agriculture_datasets = list_datasets(keyword=agriculture_keywords, usability_min=0.6, min_mb=10)

df_agriculture_datasets.head()

Lister les datasets correspondant à '['agriculture', 'food', 'farming', 'vegetable', 'livestock', 'dairy', 'poultry', 'meat', 'fruit', 'fruits', 'nuts', 'seeds', 'grains', 'cereals']' :

89 datasets trouvés.


,title,ref,description,lastUpdated,voteCount,tags,usabilityRating,dataset_size_mb,Size (Mb)
0,Agriculture crop images,aman2000jaiswal/agriculture-crop-images,,2021-03-10T16:56:50.653Z,252,"[{'ref': 'agriculture', 'name': 'agriculture',...",0.941176,59.68,59.68
1,Agriculture Crop Yield,samuelotiattakorah/agriculture-crop-yield,,2024-09-08T02:06:25.477Z,65,"[{'ref': 'agriculture', 'name': 'agriculture',...",1.000000,33.42,33.42
2,Global Food & Agriculture Statistics,unitednations/global-food-agriculture-statistics,,2017-11-16T19:24:27.387Z,222,"[{'ref': 'business', 'name': 'business', 'desc...",0.794118,443.95,443.95
3,Food Images (Food-101),kmader/food41,,2018-05-15T07:40:37.080Z,669,"[{'ref': 'popular culture', 'name': 'popular c...",0.812500,5430.04,5430.04
4,Open Food Facts,openfoodfacts/world-food-facts,,2017-09-18T12:27:58.027Z,1232,"[{'ref': 'alcohol', 'name': 'alcohol', 'descri...",0.764706,108.77,108.77


In [12]:
df_consulting_datasets.head()

,title,ref,description,lastUpdated,voteCount,tags,usabilityRating,dataset_size_mb,Size (Mb)
0,Vehicle Sales Data,syedanwarafridi/vehicle-sales-data,,2024-02-21T20:16:17.790Z,584,"[{'ref': 'transportation', 'name': 'transporta...",1.000000,18.84,18.84
1,Telecom customer,abhinav89/telecom-customer,,2017-08-27T03:01:50.747Z,207,"[{'ref': 'business', 'name': 'business', 'desc...",0.705882,13.57,13.57
2,Customer Support on Twitter,thoughtvector/customer-support-on-twitter,,2017-12-03T23:44:27.157Z,495,"[{'ref': 'business', 'name': 'business', 'desc...",0.911765,168.58,168.58
3,US Funds dataset from Yahoo Finance,stefanoleone992/mutual-funds-and-etfs,,2021-12-11T16:06:22.453Z,253,"[{'ref': 'united states', 'name': 'united stat...",1.000000,353.34,353.34
4,Massive Yahoo Finance Dataset,iveeaten3223times/massive-yahoo-finance-dataset,,2023-11-29T17:24:31.693Z,52,"[{'ref': 'research', 'name': 'research', 'desc...",1.000000,22.78,22.78


In [ ]:
url_list = []
metadata_list = []
count = 0

import os
import subprocess
import time
from tqdm import tqdm

def check_kaggle_auth():
    """Vérifie si l'utilisateur est bien authentifié avec l'API Kaggle."""
    if not os.environ.get('KAGGLE_USERNAME') or not os.environ.get('KAGGLE_KEY'):
        print("❌ Erreur : L'utilisateur n'est pas authentifié avec Kaggle. Vérifie ta configuration Kaggle.")
        return False
    return True

def download_dataset(dataset_ref: str, path: str = ".", dataset_size=None):
    """
    Télécharge un dataset Kaggle avec une barre de progression.
    
    Args:
        dataset_ref (str): Référence Kaggle, ex. "username/dataset-name".
        path (str, optionnel): Répertoire d’enregistrement.
        dataset_size (float, optionnel): Taille en Mo, pour la barre de progression.
    """
    if not check_kaggle_auth():
        return  # Stoppe la fonction si Kaggle n'est pas authentifié
    
    dataset_title = dataset_ref.split("/")[-1]  # Extraire le titre du dataset de la référence
    dataset_folder = os.path.join(path, dataset_title)  # Chemin complet pour le dataset

    # Créer le dossier si il n'existe pas déjà
    if not os.path.exists(dataset_folder):
        try:
            os.makedirs(dataset_folder, exist_ok=True)  # Crée le dossier de destination
            print(f"📁 Création du dossier : '{dataset_folder}'")
        except Exception as e:
            print(f"❌ Impossible de créer le dossier '{dataset_folder}'. Erreur : {str(e)}")
            return  # Arrêter l'exécution si on ne peut pas créer le dossier

    # Commande de téléchargement via Kaggle CLI
    command = ["kaggle", "datasets", "download", "-d", dataset_ref, "-p", dataset_folder, "--unzip"]

    # Afficher la taille du dataset si elle est fournie
    if dataset_size:
        print(f"📥 Téléchargement de '{dataset_title}' ({dataset_size:.2f} MB)...")
    else:
        print(f"📥 Téléchargement de '{dataset_title}'...")

    try:
        # Lancer le processus de téléchargement
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Si la taille du dataset est connue, afficher la progression
        with tqdm(total=dataset_size, unit="MB", unit_scale=True, desc="📥 Progression") as pbar:
            while process.poll() is None:
                time.sleep(1)
                pbar.update(1)

        # Vérifier si le téléchargement a réussi
        if process.returncode == 0:
            print(f"✅ Dataset '{dataset_title}' téléchargé avec succès dans '{dataset_folder}'.")
            # Renommage du fichier téléchargé avec la taille
            downloaded_file = os.path.join(dataset_folder, dataset_title + ".zip")
            if os.path.exists(downloaded_file):
                size_str = f"{dataset_size:.2f}" if dataset_size < 1000 else f"{dataset_size / 1000:.2f}GB"
                new_name = os.path.join(dataset_folder, f"{dataset_title}_{size_str}.zip")
                os.rename(downloaded_file, new_name)
                print(f"📦 Fichier renommé : {new_name}")
        else:
            print(f"❌ Erreur lors du téléchargement de '{dataset_title}'. Code de retour : {process.returncode}")
            print(f"💡 Détails de l'erreur : {process.stderr.read().decode('utf-8')}")

    except subprocess.CalledProcessError as e:
        print(f"❌ Erreur lors du téléchargement. Code de retour : {e.returncode}")
        print(f"💡 Détails : {e.stderr.decode('utf-8')}")
    except FileNotFoundError:
        print("❌ Erreur : La commande 'kaggle' n'a pas été trouvée. Assurez-vous que l'API Kaggle est installée.")
    except Exception as e:
        print(f"❌ Une erreur inattendue est survenue : {str(e)}")

# Exemple d'utilisation
download_path = "downloaded_datasets"

for idx, row in df_consulting_datasets.iterrows():
    ref = row["ref"]
    size_mb = row.get("Size (Mb)", None)  # Sécurisé même si la colonne est absente

    try:
        download_dataset(ref, path=download_path, dataset_size=size_mb)
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement de {ref} : {str(e)}\n")
        continue

# Exemple d'utilisation
download_path = "downloaded_datasets"

for idx, row in df_consulting_datasets.iterrows():
    ref = row["ref"]
    size_mb = row.get("Size (Mb)", None)  # Sécurisé même si la colonne est absente

    try:
        download_dataset(ref, path=download_path, dataset_size=size_mb)
        print(f"✅ Dataset '{ref}' téléchargé avec succès.\n")
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement de {ref} : {str(e)}\n")
        continue

📥 Téléchargement de 'vehicle-sales-data' (18.84 MB)...


📥 Progression:  21%|██        | 4.00/18.8 [00:04<00:14, 1.00s/MB]


✅ Dataset 'vehicle-sales-data' téléchargé avec succès dans 'downloaded_datasets\vehicle-sales-data'.
✅ Dataset 'syedanwarafridi/vehicle-sales-data' téléchargé avec succès.

📥 Téléchargement de 'telecom-customer' (13.57 MB)...


📥 Progression:  29%|██▉       | 4.00/13.6 [00:04<00:09, 1.00s/MB]


✅ Dataset 'telecom-customer' téléchargé avec succès dans 'downloaded_datasets\telecom-customer'.
✅ Dataset 'abhinav89/telecom-customer' téléchargé avec succès.

📥 Téléchargement de 'customer-support-on-twitter' (168.58 MB)...


...

📥 Progression:  13%|█▎        | 36.0/276 [00:36<04:00, 1.00s/MB]


✅ Dataset 'russia-real-estate-2021' téléchargé avec succès dans 'downloaded_datasets\russia-real-estate-2021'.
✅ Dataset 'mrdaniilak/russia-real-estate-2021' téléchargé avec succès.

📥 Téléchargement de 'japan-real-estate-transaction-prices' (97.31 MB)...


📥 Progression:  17%|█▋        | 17.0/97.3 [00:17<01:20, 1.00s/MB]

✅ Dataset 'japan-real-estate-transaction-prices' téléchargé avec succès dans 'downloaded_datasets\japan-real-estate-transaction-prices'.
✅ Dataset 'nishiodens/japan-real-estate-transaction-prices' téléchargé avec succès.



: 

Renaming Datasets with Size Information

In [ ]:
def rename_files_with_size(directory):
    """
    Parcours un dossier et tous ses sous-dossiers, et renomme tous les fichiers de données (CSV, Excel, JSON, etc.)
    en ajoutant leur taille à la fin du nom du fichier (avant l'extension).
    
    Args:
        directory (str): Répertoire où chercher les fichiers.
    """
    # Extensions des fichiers de données à vérifier
    valid_extensions = ['.csv', '.xml', '.xlsx', '.xls', '.json', '.txt', '.parquet', '.tsv']
    
    # Parcours du répertoire et de ses sous-dossiers
    for root, dirs, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            
            if any(filename.endswith(ext) for ext in valid_extensions):
                file_size = os.path.getsize(file_path) / (1024 * 1024)  # Taille en Mo
                
                # Si la taille dépasse 1000 MB, on la convertit en GB
                if file_size > 1000:
                    size_str = f"{file_size / 1024:.2f}GB"
                else:
                    size_str = f"{file_size:.2f}MB"
                
                # Sépare le nom du fichier et son extension
                name, ext = os.path.splitext(filename)
                
                # Nouveau nom avec la taille ajoutée
                new_filename = f"{name}_{size_str}{ext}"
                new_file_path = os.path.join(root, new_filename)
                
                # Renommage du fichier
                os.rename(file_path, new_file_path)


Downloading Agriculture datasets...

In [ ]:
download_path = "downloaded_agriculture_datasets"

for idx, row in df_consulting_datasets.iterrows():
    ref = row["ref"]
    size_mb = row.get("Size (Mb)", None)  # Sécurisé même si la colonne est absente

    try:
        download_dataset(ref, path=download_path, dataset_size=size_mb)
        print(f"✅ Dataset '{ref}' téléchargé avec succès.\n")
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement de {ref} : {str(e)}\n")
        continue

Functions For Models

In [ ]:
def list_models(keyword):
    print(f"Lister les modèles correspondant à '{keyword}' :")
    
    api = KaggleApi()
    api.authenticate()
    
    models = api.models_list(search=keyword)
    desired_columns = [
        'id', 'ref', 'title', 'description', 'url',
        'totalVotes', 'lastUpdated', 'tags', 'licenseName'
    ]
    
    result = []
    for model in models:
        model_dict = {key: model[key] for key in model if key in desired_columns}
        result.append(model_dict)
    
    df = pd.DataFrame(result)

    # Suppression des colonnes inutiles contenant "has" ou "Nullable"
    df = df.loc[:, ~df.columns.str.contains('has|Nullable', case=False)]
    df["tags"] = df["tags"].apply(lambda x: [d["name"] for d in x] if x is not None else [])
    # Vérification des liens URL pour voir si les modèles sont accessibles
    for idx, row in df.iterrows():
        url = row["url"]
        try:
            response = requests.get(url)
            if response.status_code >= 400:
                df.drop(idx, inplace=True)
        except requests.exceptions.RequestException:
            df.drop(idx, inplace=True)
    

    return df